In [ ]:
import random
import pandas as pd

In [ ]:
df = pd.read_csv("Event_Table.csv")

In [ ]:
df.head(20)

,Trust_Event_Id,Frame_Id,Event_Id,Trustee,Trustor,Is_One_Hop,Event_Type,Has_Dropped,Has_Modified,MDR,MMR,Type_MMR,Type_MDR
0,1,1,1,17,9999,0,2,0,0,0.0,0.0,0.0,0.0
1,2,1,2,9,1,1,3,0,0,0.0,0.0,0.0,0.0
2,3,1,2,9,6,1,3,0,0,0.0,0.0,0.0,0.0
3,4,1,2,9,8,1,3,0,0,0.0,0.0,0.0,0.0
4,5,1,2,9,11,1,3,0,0,0.0,0.0,0.0,0.0
5,6,1,2,9,26,1,3,0,0,0.0,0.0,0.0,0.0
6,7,1,2,9,28,1,3,0,0,0.0,0.0,0.0,0.0
7,8,1,2,1,25,1,3,0,0,0.0,0.0,0.0,0.0
8,9,1,2,8,4,1,3,0,0,0.0,0.0,0.0,0.0
9,10,1,2,8,5,1,3,0,0,0.0,0.0,0.0,0.0


In [ ]:
df[(df["Trustee"]==11) & (df["Trustor"]==16)]

,Trust_Event_Id,Frame_Id,Event_Id,Trustee,Trustor,Is_One_Hop,Event_Type,Has_Dropped,Has_Modified,MDR,MMR,Type_MMR,Type_MDR
6952,6953,162,310,11,16,1,3,0,0,2.898551,3.431373,3.500000,1.851852
6985,6986,162,311,11,16,1,3,0,0,2.898551,3.431373,3.500000,1.851852
7010,7011,163,312,11,16,1,3,0,0,2.857143,3.341289,3.317536,1.818182
7042,7043,163,313,11,16,1,3,0,0,2.857143,3.341289,3.317536,1.818182
7076,7077,164,314,11,16,1,3,0,0,2.816901,3.309693,3.255814,1.785714
7205,7206,166,318,11,16,1,2,0,0,2.739726,3.964758,4.186047,4.000000
7222,7223,166,319,11,16,1,3,0,1,2.739726,3.964758,3.765690,1.724138
7401,7402,167,324,11,16,1,2,0,0,2.702703,3.958333,4.504505,3.846154
7418,7419,167,325,11,16,1,3,0,0,2.702703,3.958333,3.488372,1.694915
7685,7686,168,332,11,16,1,2,0,0,2.666667,3.853955,4.366812,3.703704


In [ ]:
def get_base_likelihood(trustee, base_likelihoods):
    """
    Return (p, q) for the given trustee.
    If the trustee doesn't exist in base_likelihoods, generate random values and store them.
    """
    if trustee not in base_likelihoods:
        # p = Probability of positive recommendation if X is trustworthy
        # q = Probability of positive recommendation if X is untrustworthy
        p = random.uniform(0.6, 0.8)  # between 60% and 90%
        q = random.uniform(0.1, 0.4)  # between 10% and 40%
        base_likelihoods[trustee] = (p, q)
    return base_likelihoods[trustee]

In [ ]:
def get_modified_likelihood(trustee, modification_likelihoods, p, q):
    """
    Return (p_modified, q_modified) for the given trustee.
    If not already in modification_likelihoods, generate new values (or derive from p,q).
    """
    if trustee not in modification_likelihoods:
        # - Decrease p by a random factor to simulate attacker lowering the chance of a positive recommedation
        # - Increase q by a random factor to simulate attacker raising the chance of a positive reccomendation
        p_mod = p * random.uniform(0.5, 0.8)  # reduce p by 20%~50%
        q_mod = q + random.uniform(0.1, 0.3)  # increase q by 0.1~0.3
        # To Make sure q_mod <= 1.0
        q_mod = min(q_mod, 1.0)

        modification_likelihoods[trustee] = (p_mod, q_mod)
    return modification_likelihoods[trustee]

In [ ]:
def bayesian_trust_update(data,
                          prior_trust=0.6,
                          prior_not_trust=0.4):
    """
    data: Pandas DataFrame with columns:
          [Trustee, Trustor, Has_Dropped, Has_Modified, ...]
    prior_trust: P(T)
    prior_not_trust: P(~T)

    Returns a dict with keys = (trustor, trustee), values = final posterior P(T|E).
    """
    # For numerical stability, ensure priors sum to 1
    assert abs(prior_trust + prior_not_trust - 1.0) < 1e-9, \
        "Prior probabilities must sum to 1."

    # Dictionaries to hold partial products
    partial_likelihood_T = {}
    partial_likelihood_notT = {}

    # Dictionaries to store generated likelihoods
    base_likelihoods = {}
    modification_likelihoods = {}

    # Iterate over each event
    for idx, row in data.iterrows():
        trustee = row["Trustee"]
        trustor = row["Trustor"]
        dropped = row["Has_Dropped"]
        modified = row["Has_Modified"]

        # If message dropped, skip
        if dropped == 1:
            continue

        # Retrieve base (p, q) or create them randomly if trustee is new
        p_base, q_base = get_base_likelihood(trustee, base_likelihoods)

        if modified == 1:
            # Retrieve or generate modified (p_mod, q_mod)
            p, q = get_modified_likelihood(trustee, modification_likelihoods, p_base, q_base)
        else:
            p, q = p_base, q_base

        # Initialize partial products if not present
        if (trustor, trustee) not in partial_likelihood_T:
            partial_likelihood_T[(trustor, trustee)] = 1.0
            partial_likelihood_notT[(trustor, trustee)] = 1.0

        # Multiply the partial likelihoods
        partial_likelihood_T[(trustor, trustee)] *= p
        partial_likelihood_notT[(trustor, trustee)] *= q

    # Compute final posteriors
    results = {}
    for (trustor, trustee) in partial_likelihood_T:
        t_prod = partial_likelihood_T[(trustor, trustee)]
        nt_prod = partial_likelihood_notT[(trustor, trustee)]

        numerator = prior_trust * t_prod
        denominator = numerator + (prior_not_trust * nt_prod)

        if denominator == 0:
            posterior = 0.0
        else:
            posterior = numerator / denominator

        results[(trustor, trustee)] = posterior

    return results

In [ ]:
def main():

    # Run Bayesian trust update
    trust_results = bayesian_trust_update(df, prior_trust=0.6, prior_not_trust=0.4)

    # Print the results
    print("Final Trust Values (Posterior P(T|E)):")
    for (trustor, trustee), trust_val in trust_results.items():
        print(f"Trustor={trustor}, Trustee={trustee}, Trust={trust_val:.4f}")

In [ ]:
if __name__ == "__main__":
    main()

Final Trust Values (Posterior P(T|E)):
Trustor=9999.0, Trustee=17.0, Trust=0.9751
Trustor=1.0, Trustee=9.0, Trust=1.0000
Trustor=6.0, Trustee=9.0, Trust=1.0000
Trustor=8.0, Trustee=9.0, Trust=1.0000
Trustor=11.0, Trustee=9.0, Trust=1.0000
Trustor=26.0, Trustee=9.0, Trust=1.0000
Trustor=28.0, Trustee=9.0, Trust=0.9100
Trustor=25.0, Trustee=1.0, Trust=1.0000
Trustor=4.0, Trustee=8.0, Trust=1.0000
Trustor=5.0, Trustee=8.0, Trust=1.0000
Trustor=25.0, Trustee=11.0, Trust=1.0000
Trustor=4.0, Trustee=28.0, Trust=0.9999
Trustor=4.0, Trustee=9.0, Trust=1.0000
Trustor=5.0, Trustee=9.0, Trust=1.0000
Trustor=25.0, Trustee=9.0, Trust=1.0000
Trustor=1.0, Trustee=25.0, Trust=1.0000
Trustor=11.0, Trustee=25.0, Trust=1.0000
Trustor=6.0, Trustee=1.0, Trust=1.0000
Trustor=9.0, Trustee=1.0, Trust=1.0000
Trustor=26.0, Trustee=1.0, Trust=0.9594
Trustor=28.0, Trustee=1.0, Trust=0.8562
Trustor=9.0, Trustee=11.0, Trust=1.0000
Trustor=26.0, Trustee=11.0, Trust=0.9999
Trustor=6.0, Trustee=25.0, Trust=1.0000
Trus

In [ ]:
#Reading CSV files
df = pd.read_csv("Event_Table.csv")